https://www.guru99.com/pyspark-tutorial.html
    

https://databricks.com/blog/2020/03/31/10-minutes-from-pandas-to-koalas-on-apache-spark.html

https://databricks.com/blog/2019/04/24/koalas-easy-transition-from-pandas-to-apache-spark.html

https://databricks-prod-cloudfront.cloud.databricks.com/public/4027ec902e239c93eaaa8714f173bcfc/8266758089056472/1487766102750011/5693546805547978/latest.html

https://www.programiz.com/python-programming/regex

https://www.analyticsvidhya.com/blog/2017/02/top-28-cheat-sheets-for-machine-learning-data-science-probability-sql-big-data/

https://www.analyticsvidhya.com/blog/2016/01/12-pandas-techniques-python-data-manipulation/

https://www.guru99.com/python-regular-expressions-complete-tutorial.html

https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

https://towardsdatascience.com/pyspark-and-sparksql-basics-6cb4bf967e53

https://sparkbyexamples.com/spark/spark-collect-list-and-collect-set-functions/


https://gist.github.com/bsweger/e5817488d161f37dcbd2  useful pandas snippets

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os 
import datetime
from requests import get
from os import path, getcwd, makedirs, listdir
from io import BytesIO
from zipfile import ZipFile
import timeit

#pip install koalas




In [2]:
#os.environ
os.getenv('SPARK_HOME')

'C:\\Spark\\spark-3.1.1-bin-hadoop2.7'

In [4]:
#!pip install findspark

In [3]:
import findspark
findspark.init()

In [4]:
import pyspark
#Import koalas:
import databricks.koalas as ks

In [5]:
from pyspark import SQLContext
from pyspark.sql import SparkSession 
 
conf = pyspark.SparkConf() 
 
sc = pyspark.SparkContext.getOrCreate(conf=conf)
#sqlcontext = SQLContext(sc)
spark = SparkSession(sc) 

In [6]:
os.getcwd()

'C:\\Users\\hp\\Downloads\\TEST projects'

In [5]:
os.chdir("C:\\Users\\hp\\Downloads\\TEST_projects")

In [ ]:
#create folder:
folder_data = 'data'

if not os.path.exists(folder_data):
    os.makedirs(folder_data)
else:
    print('Folder already exists')
        
#loop through all files (from url) starting at month 1 to 12:
for month in range(1,13):
    month_string = str(month)
    month_leading_zero = month_string.zfill(2)
    
    #scrape URL:                                                      +01, 02, 03...eg:201801-fordgobike-tripdata.csv.zip
    bike_data_url = 'https://s3.amazonaws.com/fordgobike-data/2018' + month_leading_zero + '-fordgobike-tripdata.csv.zip'
    response = get(bike_data_url)

    # code below opens zip file; BytesIO returns a view of the contents;
    unzipped_file = ZipFile(BytesIO(response.content))

    # place extracted zip files into folder_data
    unzipped_file.extractall(folder_data)


In [6]:
%%time

#import all csv files into a spark df: 12 files
df = spark.read.option("header", "true").csv('C:\\Users\\hp\\Downloads\\TEST projects\\data\\*.csv')

Wall time: 13.3 s


In [16]:
type(df)

pyspark.sql.dataframe.DataFrame

In [7]:
%%time

#check df:
df.show(vertical=True)

-RECORD 0---------------------------------------
 duration_sec            | 80066                
 start_time              | 2018-10-31 20:49:... 
 end_time                | 2018-11-01 19:03:... 
 start_station_id        | 138                  
 start_station_name      | Jersey St at Chur... 
 start_station_latitude  | 37.7509004           
 start_station_longitude | -122.4274114         
 end_station_id          | 147                  
 end_station_name        | 29th St at Tiffan... 
 end_station_latitude    | 37.7440667           
 end_station_longitude   | -122.4214722         
 bike_id                 | 3458                 
 user_type               | Customer             
 member_birth_year       | 1987                 
 member_gender           | Female               
 bike_share_for_all_trip | No                   
-RECORD 1---------------------------------------
 duration_sec            | 56428                
 start_time              | 2018-10-31 20:18:... 
 end_time           

In [16]:
%%time

#Read in using Koalas read_csv
kdf = ks.read_csv('C:\\Users\\hp\\Downloads\\TEST projects\\data\\*.csv', header=0, sep=",")
#this is slow, try converting to koalas from pyspark df

Wall time: 26.2 s


In [8]:
#to avoid whole partition in single node when converting spark df to koalas:
ks.set_option('compute.default_index_type', 'distributed')

In [9]:
%%time

#convert pyspark df to koalas df:
kdf = df.to_koalas() #this is efficient

Wall time: 896 ms


In [9]:
%%time

kdf.head(20) #display 20 records to make it equal to pyspark .show method

Wall time: 765 ms


,duration_sec,start_time,end_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,member_birth_year,member_gender,bike_share_for_all_trip
0,80066,2018-10-31 20:49:03.1400,2018-11-01 19:03:30.0550,138,Jersey St at Church St,37.7509004,-122.4274114,147,29th St at Tiffany Ave,37.7440667,-122.4214722,3458,Customer,1987,Female,No
1,56428,2018-10-31 20:18:02.8810,2018-11-01 11:58:31.1460,75,Market St at Franklin St,37.7737932060887,-122.42123901844025,86,Market St at Dolores St,37.7693053,-122.4268256,1798,Customer,None,None,No
2,44825,2018-10-31 20:07:13.3780,2018-11-01 08:34:19.3670,139,Garfield Square (25th St at Harrison St),37.7510171,-122.4119009,122,19th St at Mission St,37.760299,-122.418892,1575,Customer,1983,Female,No
3,55461,2018-10-31 17:06:37.6830,2018-11-01 08:30:59.2200,67,San Francisco Caltrain Station 2 (Townsend St...,37.7766392,-122.3955263,16,Steuart St at Market St,37.79413,-122.39443,2978,Subscriber,1992,Female,No
4,47724,2018-10-31 18:27:36.8230,2018-11-01 07:43:01.5610,259,Addison St at Fourth St,37.866249,-122.2993708,259,Addison St at Fourth St,37.866249,-122.2993708,2392,Customer,None,None,No
5,32818,2018-10-31 20:54:06.5050,2018-11-01 06:01:05.3020,345,Hubbell St at 16th St,37.76647421515012,-122.39829540252686,109,17th St at Valencia St,37.7633158,-122.4219039,1466,Customer,1975,Male,No
6,18888,2018-10-31 21:43:43.0640,2018-11-01 02:58:31.2400,17,Embarcadero BART Station (Beale St at Market St),37.792251,-122.397086,74,Laguna St at Hayes St,37.776434819204745,-122.42624402046204,4452,Subscriber,1995,Female,No
7,1786,2018-10-31 23:44:10.7030,2018-11-01 00:13:56.9500,323,Broadway at Kearny,37.79801364395978,-122.40595042705534,27,Beale St at Harrison St,37.7880593,-122.3918648,4050,Subscriber,1977,Male,No
8,4391,2018-10-31 23:00:13.7270,2018-11-01 00:13:25.1030,241,Ashby BART Station,37.8524766,-122.2702132,272,Woolsey St at Sacramento St,37.850577684047856,-122.27817535400389,1789,Customer,None,None,No
9,1752,2018-10-31 23:40:10.4300,2018-11-01 00:09:23.0920,81,Berry St at 4th St,37.77588,-122.39317,3,Powell St BART Station (Market St at 4th St),37.78637526861584,-122.40490436553954,1525,Subscriber,1986,Other,Yes


In [12]:
%%time

kdf.shape #quick

Wall time: 2.32 s


(1863721, 16)

In [10]:
%%time

#cols and rows in pyspark:
print((df.count(), len(df.columns)))

(1863721, 16)
Wall time: 2.78 s


In [11]:
%%time

#cols ps:
df.columns

Wall time: 0 ns


['duration_sec',
 'start_time',
 'end_time',
 'start_station_id',
 'start_station_name',
 'start_station_latitude',
 'start_station_longitude',
 'end_station_id',
 'end_station_name',
 'end_station_latitude',
 'end_station_longitude',
 'bike_id',
 'user_type',
 'member_birth_year',
 'member_gender',
 'bike_share_for_all_trip']

In [13]:
%%time 

#cols kdf:
kdf.columns

Wall time: 0 ns


Index(['duration_sec', 'start_time', 'end_time', 'start_station_id',
       'start_station_name', 'start_station_latitude',
       'start_station_longitude', 'end_station_id', 'end_station_name',
       'end_station_latitude', 'end_station_longitude', 'bike_id', 'user_type',
       'member_birth_year', 'member_gender', 'bike_share_for_all_trip'],
      dtype='object')

In [14]:
%%time

#cols and dtypes ps:
df.printSchema()

root
 |-- duration_sec: string (nullable = true)
 |-- start_time: string (nullable = true)
 |-- end_time: string (nullable = true)
 |-- start_station_id: string (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_latitude: string (nullable = true)
 |-- start_station_longitude: string (nullable = true)
 |-- end_station_id: string (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_latitude: string (nullable = true)
 |-- end_station_longitude: string (nullable = true)
 |-- bike_id: string (nullable = true)
 |-- user_type: string (nullable = true)
 |-- member_birth_year: string (nullable = true)
 |-- member_gender: string (nullable = true)
 |-- bike_share_for_all_trip: string (nullable = true)

Wall time: 4.5 ms


In [15]:
%%time

kdf.info()

<class 'databricks.koalas.frame.DataFrame'>
Int64Index: 1863721 entries, 0 to 1863720
Data columns (total 16 columns):
 #   Column                   Non-Null Count    Dtype 
---  ------                   --------------    ----- 
 0   duration_sec             1863721 non-null  object
 1   start_time               1863721 non-null  object
 2   end_time                 1863721 non-null  object
 3   start_station_id         1863721 non-null  object
 4   start_station_name       1863721 non-null  object
 5   start_station_latitude   1863721 non-null  object
 6   start_station_longitude  1863721 non-null  object
 7   end_station_id           1863721 non-null  object
 8   end_station_name         1863721 non-null  object
 9   end_station_latitude     1863721 non-null  object
 10  end_station_longitude    1863721 non-null  object
 11  bike_id                  1863721 non-null  object
 12  user_type                1863721 non-null  object
 13  member_birth_year        1753003 non-null  object
 

In [10]:
%%time

#rename cols ps:
df = df.withColumnRenamed("duration_sec", "dur_sec")\
       .withColumnRenamed("end_station_latitude", "end_station_lat")\
       .withColumnRenamed("end_station_longitude", "end_station_long")

Wall time: 49.9 ms


In [11]:
%%time

kdf = kdf.rename(columns = {"duration_sec": "dur_sec", "end_station_latitude": "end_station_lat",
                      "end_station_longitude": "end_station_long"})

Wall time: 894 ms


In [12]:
%%time

from pyspark.sql.functions import col
#create new col ps:
df = df.withColumn('member_age', 2018 - col('member_birth_year'))
#change new col to int:
df = df.withColumn('member_age', df.member_age.cast('int'))

Wall time: 1.56 s


In [21]:
#test:
df.printSchema()

root
 |-- dur_sec: string (nullable = true)
 |-- start_time: string (nullable = true)
 |-- end_time: string (nullable = true)
 |-- start_station_id: string (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_latitude: string (nullable = true)
 |-- start_station_longitude: string (nullable = true)
 |-- end_station_id: string (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_lat: string (nullable = true)
 |-- end_station_long: string (nullable = true)
 |-- bike_id: string (nullable = true)
 |-- user_type: string (nullable = true)
 |-- member_birth_year: string (nullable = true)
 |-- member_gender: string (nullable = true)
 |-- bike_share_for_all_trip: string (nullable = true)
 |-- member_age: integer (nullable = true)



In [11]:
kdf.columns

Index(['dur_sec', 'start_time', 'end_time', 'start_station_id',
       'start_station_name', 'start_station_latitude',
       'start_station_longitude', 'end_station_id', 'end_station_name',
       'end_station_lat', 'end_station_long', 'bike_id', 'user_type',
       'member_birth_year', 'member_gender', 'bike_share_for_all_trip'],
      dtype='object')

In [85]:
kdf.isnull().sum()

dur_sec                         0
start_time                      0
end_time                        0
start_station_id                0
start_station_name              0
start_station_latitude          0
start_station_longitude         0
end_station_id                  0
end_station_name                0
end_station_lat                 0
end_station_long                0
bike_id                         0
user_type                       0
member_birth_year          110718
member_gender              110367
bike_share_for_all_trip         0
member_age                 110718
dtype: int64

In [ ]:
#when calculating the age column, kdf returns float ints, whereas ps returns whole numbers. This is the desired format going
#forward as trailing decimals will cause issues when doing computations or aggregations.


#NOTE:
#there are NaNs in age col and pandas/koalas needs to use a float64 type to represent NaNs -> this is why the conversion to
# int returns decimal point(.0)
#SOLUTION -> drop nan values in year col before age calculation in order to return whole numbers

#this is one massive disadvantage in comparison to age calc in pyspark df --> see possible solution below

In [13]:
#drop nans in year col before age calculation:
kdf.dropna(subset=['member_birth_year'], inplace = True)

In [89]:
#test:
kdf.isnull().sum()

dur_sec                    0
start_time                 0
end_time                   0
start_station_id           0
start_station_name         0
start_station_latitude     0
start_station_longitude    0
end_station_id             0
end_station_name           0
end_station_lat            0
end_station_long           0
bike_id                    0
user_type                  0
member_birth_year          0
member_gender              0
bike_share_for_all_trip    0
member_age                 0
dtype: int64

In [14]:
%%time

#new col:
kdf['member_age'] = 2018 - kdf['member_birth_year'].astype(int)
kdf.member_age.dtype

Wall time: 341 ms


dtype('int32')

In [13]:
%%time

#summary stats age in ps:
df.describe(['member_age']).show()

+-------+------------------+
|summary|        member_age|
+-------+------------------+
|  count|           1753003|
|   mean| 34.91203780027758|
| stddev|10.442886209686751|
|    min|                18|
|    max|               137|
+-------+------------------+

Wall time: 30.5 s


In [96]:
%%time

#summary stats age in kdf:
kdf.describe() #why return of floats when age is whole number in df?

Wall time: 2.97 s


,member_age
count,1.753003e+06
mean,3.491204e+01
std,1.044289e+01
min,1.800000e+01
25%,2.700000e+01
50%,3.300000e+01
75%,4.000000e+01
max,1.370000e+02


In [ ]:
#NOTE:

#dropping nan values in year col before calc of age col does return whole numbers to each row of df, however, when displaying
#stats with .describe(), the return values are still floats, which is so flippen annoying -_-


## posible solution:
# use pd.options.display.float_format = "{:.0f}".format  --> do not know the possible consequences\exprnse
#of run time with this

In [13]:
#try panadas method:
import pandas as pd
pd.options.display.float_format = "{:.0f}".format

In [15]:
%%time

# run describe again:
kdf.describe() # now whole numbers are returned instead of floats when pd.option is used

#does not seem to affect run time as runtime is still quicker that pysparkDF.describe()

Wall time: 3.62 s


,member_age
count,1753003
mean,35
std,10
min,18
25%,27
50%,33
75%,40
max,137


In [15]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *
from pyspark.sql import functions as F

In [17]:
#need to drop nan values in order for the below condition to work:
df.na.drop(subset=["member_age"])

DataFrame[dur_sec: string, start_time: string, end_time: string, start_station_id: string, start_station_name: string, start_station_latitude: string, start_station_longitude: string, end_station_id: string, end_station_name: string, end_station_lat: string, end_station_long: string, bike_id: string, user_type: string, member_birth_year: string, member_gender: string, bike_share_for_all_trip: string, member_age: int]

In [16]:
%%time

#create age_category [new_col] ps:
df = df.withColumn(
    'age_group',
    F.when(F.col('member_age').between(18, 25), 'young adult')\
    .when(F.col('member_age').between(26, 40), 'adult')\
    .when(F.col('member_age').between(41, 60), 'middle age')\
    .when((F.col('member_age') > 61), 'mature adult')\
    .otherwise(None)
)

Wall time: 105 ms


In [16]:
%%time

#test:
df.select('age_group').show()

+-----------+
|  age_group|
+-----------+
|      adult|
|       null|
|      adult|
|      adult|
|       null|
| middle age|
|young adult|
| middle age|
|       null|
|      adult|
|young adult|
|      adult|
|       null|
|young adult|
|      adult|
|      adult|
|young adult|
|young adult|
|young adult|
|young adult|
+-----------+
only showing top 20 rows

Wall time: 487 ms


In [ ]:
#why does this age group col have nuls? It shouldn't because all na's were dropped prior to creating the bins.

In [20]:
#nulls in age group col:
#df.filter(df.age_group.isNotNull()).count()
df.filter('age_group is NULL').count() #this is odd because na's were dropped in age col

116686

In [134]:
%%time

df.filter('member_age is NULL').count() #-_-

Wall time: 2.38 s


110718

In [37]:
#set options for below function to run successfully:
ks.set_option('compute.ops_on_diff_frames', True)  #--> dont do this

In [23]:
%%time

#create age group col for kdf with np --> this does not perform well with koalas df(extremely long runtime ~ >5min)

# --> Try if:
def condition(i):
    if (i['member_age'] >= 18) & (i['member_age'] <= 25):
        return 'young_adult'
    elif (i['member_age'] >= 26) & (i['member_age'] <= 40):
        return 'adult'
    elif (i['member_age'] >= 41) & (i['member_age'] <= 60):
        return 'middle age'
    elif (i['member_age'] > 61):
        return 'mature adult'
    else:
        return None
    
kdf['age_group'] = kdf.apply(condition, axis= 1)

#this method has a significantly longer run time that ps

Wall time: 23.2 s


In [17]:
%%time

#try sql statement to see if runtime is quicker:  == it is

kdf2 = ks.sql("select *, \
       CASE \
       WHEN (member_age >= 18 AND member_age <=25) THEN 'young_adult' \
       WHEN (member_age >= 26 AND member_age <=40) THEN 'adult' \
       WHEN (member_age >= 41 AND member_age <=60) THEN 'middle age' \
       ELSE 'mature adult' \
       END AS age_group \
       from {kdf}") #wrap a DataFrame with curly braces to query it directly

# OR to koalas:
#kdf2 = ks.read_sql(query) and then join to kdf on bike_id -> can't join, bike ids are not unique

Wall time: 1.99 s


In [24]:
kdf2.head(3)

,dur_sec,start_time,end_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_lat,end_station_long,bike_id,user_type,member_birth_year,member_gender,bike_share_for_all_trip,member_age,age_group
0,80066,2018-10-31 20:49:03.1400,2018-11-01 19:03:30.0550,138,Jersey St at Church St,37.7509004,-122.4274114,147,29th St at Tiffany Ave,37.7440667,-122.4214722,3458,Customer,1987,Female,No,31,adult
1,44825,2018-10-31 20:07:13.3780,2018-11-01 08:34:19.3670,139,Garfield Square (25th St at Harrison St),37.7510171,-122.4119009,122,19th St at Mission St,37.760299,-122.418892,1575,Customer,1983,Female,No,35,adult
2,55461,2018-10-31 17:06:37.6830,2018-11-01 08:30:59.2200,67,San Francisco Caltrain Station 2 (Townsend St...,37.7766392,-122.3955263,16,Steuart St at Market St,37.79413,-122.39443,2978,Subscriber,1992,Female,No,26,adult


In [26]:
%%time

#test:
kdf2[['age_group']].head()

Wall time: 95.5 ms


,age_group
0,adult
2,adult
3,adult
5,middle age
6,young_adult


In [34]:
%%time

#value counts ps:
df.groupby("age_group").count().orderBy('count', ascending = False).show()

+------------+-------+
|   age_group|  count|
+------------+-------+
|       adult|1031867|
|  middle age| 391699|
| young adult| 292741|
|        null| 116686|
|mature adult|  30728|
+------------+-------+

Wall time: 3.22 s


In [46]:
df.groupby("user_type").count().orderBy('count', ascending = False).show()

+----------+-------+
| user_type|  count|
+----------+-------+
|Subscriber|1583554|
|  Customer| 280167|
+----------+-------+



In [25]:
%%time

#value_counts kdf:
kdf2.age_group.value_counts()

Wall time: 113 ms


adult           1031867
middle age       391699
young_adult      292741
mature adult      36696
Name: age_group, dtype: int64

In [19]:
%%time

#prepare for join time comparison:
#use sql query to create new spark df and then do a join to combine new col to existing df:
#df.createOrReplaceTempView("temp")

#df2 = spark.sql("""select bike_id, CASE WHEN user_type = 'Customer' THEN 'yes' ELSE 'no' END AS is_member from temp""")
#code has to be in one line otherwise an error is thrown

Wall time: 60.8 ms


In [18]:
%%time

#inspect new df:
#df2.show()

Wall time: 0 ns


In [35]:
#spark.catalog.dropTempView("temp")

#DO NOT JOIN: BIKE ID's ARE NOT UNIQUE.

In [58]:
%%time

#inspect df:
#kd.head(20)

Wall time: 88.9 ms


,bike_id,is_member
0,3458,yes
1,1575,yes
2,2978,no
3,1466,yes
4,4452,no
5,4050,no
6,1525,no
7,4436,no
8,2103,no
9,254,no


In [43]:
#kdf.bike_id.nunique()

5054

In [19]:
#test:
#df.groupby("is_member").count().orderBy('count', ascending = False).show()
#df has more counts than kdf because nulls where not sucessfully removed from ps(df)

In [20]:
%%time

#df.select('age_group', 'member_age', 'is_member').show()

Wall time: 0 ns


In [21]:
#test:
#kdf.is_member.value_counts()

In [22]:
%%time

#kdf[['age_group', 'member_age', 'is_member']].head()

Wall time: 0 ns


In [ ]:
%%time

#calculare distance between start station and end station:


In [18]:
%%time

#create a duration minutes col ps:
df = df.withColumn('dur_min', col('dur_sec') / 60)
#change new col to float:
df = df.withColumn('dur_min', df.dur_min.cast('float'))

Wall time: 37.9 ms


In [43]:
%%time

#create a duration minutes col kdf:
#convert dur_min to int:
kdf['dur_sec'] = kdf['dur_sec'].astype(float)
kdf['dur_min'] = kdf['dur_sec'] / 60

Wall time: 445 ms


In [44]:
kdf.dur_min.dtype

dtype('float64')

In [20]:
%%time

#change cols to timestamp:
from pyspark.sql.types import DateType
df = df.withColumn('start_dt', df['start_time'].cast(DateType()))

Wall time: 73.6 ms


In [28]:
df.select('start_time', 'start_dt', 'dur_min').show()

+--------------------+----------+---------+
|          start_time|  start_dt|  dur_min|
+--------------------+----------+---------+
|2018-10-31 20:49:...|2018-10-31|1334.4333|
|2018-10-31 20:18:...|2018-10-31| 940.4667|
|2018-10-31 20:07:...|2018-10-31| 747.0833|
|2018-10-31 17:06:...|2018-10-31|   924.35|
|2018-10-31 18:27:...|2018-10-31|    795.4|
|2018-10-31 20:54:...|2018-10-31| 546.9667|
|2018-10-31 21:43:...|2018-10-31|    314.8|
|2018-10-31 23:44:...|2018-10-31|29.766666|
|2018-10-31 23:00:...|2018-10-31|73.183334|
|2018-10-31 23:40:...|2018-10-31|     29.2|
|2018-10-31 23:58:...|2018-10-31| 9.616667|
|2018-10-31 23:53:...|2018-10-31|11.483334|
|2018-10-31 23:51:...|2018-10-31| 9.716666|
|2018-10-31 23:57:...|2018-10-31|4.0833335|
|2018-10-31 23:46:...|2018-10-31|    12.75|
|2018-10-31 23:43:...|2018-10-31|12.933333|
|2018-10-31 23:38:...|2018-10-31|17.683332|
|2018-10-31 23:53:...|2018-10-31|     2.85|
|2018-10-31 23:38:...|2018-10-31|17.383333|
|2018-10-31 23:39:...|2018-10-31

In [21]:
%%time

#change dtype in kdf:
kdf['start_time'] = ks.to_datetime(kdf['start_time'])

C:\Spark\spark-3.1.1-bin-hadoop2.7\python\pyspark\sql\pandas\functions.py:392: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  "in the future releases. See SPARK-28264 for more details.", UserWarning)


Wall time: 1.08 s


In [27]:
#test:
kdf.start_time.dtype

dtype('<M8')

In [30]:
#test:
kdf.head(3)

,dur_sec,start_time,end_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_lat,end_station_long,bike_id,user_type,member_birth_year,member_gender,bike_share_for_all_trip,member_age,dur_min
0,80066,2018-10-31 20:49:03.140,2018-11-01 19:03:30.0550,138,Jersey St at Church St,37.7509004,-122.4274114,147,29th St at Tiffany Ave,37.7440667,-122.4214722,3458,Customer,1987,Female,No,31,1334.433333
1,44825,2018-10-31 20:07:13.378,2018-11-01 08:34:19.3670,139,Garfield Square (25th St at Harrison St),37.7510171,-122.4119009,122,19th St at Mission St,37.760299,-122.418892,1575,Customer,1983,Female,No,35,747.083333
2,55461,2018-10-31 17:06:37.683,2018-11-01 08:30:59.2200,67,San Francisco Caltrain Station 2 (Townsend St...,37.7766392,-122.3955263,16,Steuart St at Market St,37.79413,-122.39443,2978,Subscriber,1992,Female,No,26,924.350000


In [21]:
%%time

#extract weekday from start_time in kdf:
kdf['weekday'] = kdf['start_time'].dt.weekday

Wall time: 320 ms


In [32]:
#test:
kdf['weekday'].value_counts() #0: monday - 6:sunday

1    304791
2    302138
3    299465
0    280420
4    274484
5    155466
6    136239
Name: weekday, dtype: int64

In [22]:
%%time

#try get weekday name for koalas df:
kdf['dow'] = kdf['start_time'].dt.day_name()

Wall time: 299 ms


In [59]:
kdf['dow'].value_counts()

Tuesday      304791
Wednesday    302138
Thursday     299465
Monday       280420
Friday       274484
Saturday     155466
Sunday       136239
Name: dow, dtype: int64

In [23]:
%%time

#from pyspark.sql.functions import dayofweek #use this package for weekday numeric representaion.

from pyspark.sql.functions import date_format
#extract weekday name from start_dt in ps:
df = df.withColumn('weekday',date_format(col("start_time"), "EEEE"))

Wall time: 19.9 ms


In [53]:
df.select('weekday').dtypes

[('weekday', 'string')]

In [56]:
#test:
df.groupBy('weekday').count().orderBy('count', ascending = False).show()

+---------+------+
|  weekday| count|
+---------+------+
|  Tuesday|319332|
|Wednesday|316374|
| Thursday|314124|
|   Monday|294573|
|   Friday|290390|
| Saturday|175614|
|   Sunday|153314|
+---------+------+



In [24]:
%%time

#use groupby to aggregate data by mean and append it to kdf:
kdf_mean = kdf.groupby('bike_id')['dur_min'].mean()

Wall time: 222 ms


In [64]:
%%time

#test:
kdf_mean.head(20)

Wall time: 66.8 ms


bike_id
1159    11.160152
1572    11.281767
829     11.882234
2136    13.294210
691     12.223280
2069    12.857402
3210    14.305975
1436    12.110904
3606    10.240434
2088    13.918226
1090    12.188984
296     11.300552
1512    14.548297
2904    14.473254
3414    13.786477
467     14.422379
3959    11.391667
4032    18.876368
675     10.362865
4821    12.435714
Name: dur_min, dtype: float64

In [25]:
%%time

#average duration min for each bike ride ps:
ps_mean = df.groupBy("bike_id").mean( "dur_min") 

Wall time: 24.9 ms


In [66]:
%%time

#test:
ps_mean.show()

+-------+------------------+
|bike_id|      avg(dur_min)|
+-------+------------------+
|   1159|11.888165257558102|
|   1572|11.366237925946521|
|    829|12.824093171351931|
|   2136|14.786178128883757|
|    691|13.902967401711098|
|   2069|15.484368357670473|
|   3210|15.126336506093448|
|   1436|15.730126621451559|
|   3606|10.515544849232985|
|   2088|15.730566895900129|
|   1090|14.019126026077968|
|    296|12.746385517608688|
|   1512|15.077571417263576|
|   2904|15.723533301030175|
|   3414|15.505172469287084|
|    467| 16.10021366064365|
|   3959|11.535706902805128|
|   4032|22.282628939185344|
|    675|10.707062197943864|
|   4821|12.093939372084357|
+-------+------------------+
only showing top 20 rows

Wall time: 6.23 s


In [26]:
%%time

#join the mean calc to kdf:
kdf = kdf.merge(kdf_mean)

Wall time: 1.98 s


In [27]:
%%time

#join the mean cals to ps:
df = df.join(ps_mean, 'bike_id', 'inner')

Wall time: 60.8 ms


In [28]:
%%time

#calculate distance:
import math
def distance(origin1, origin2, dest1, dest2):
    """
    Parameters
    ----------
    origin : tuple of float-->start station
        (lat, long)
    destination : tuple of float-->end station
        (lat, long)

    Returns
    -------
    distance_in_km : float
    """
    lat1 = origin1
    lon1 = origin2
    lat2 = dest1
    lon2 = dest2
    radius = 6371  # km

    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = (math.sin(dlat / 2) * math.sin(dlat / 2) +
         math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) *
         math.sin(dlon / 2) * math.sin(dlon / 2))
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    d = radius * c

    return d

Wall time: 0 ns


In [29]:
%%time

#long and lat to string:  --> have to convert to string first for function to execute succesfully

cols = ['start_station_longitude','start_station_latitude', 'end_station_long','end_station_lat']
for col_name in cols:
    df = df.withColumn(col_name, col(col_name).cast('float'))

Wall time: 52.9 ms


In [30]:
%%time

#Create udf to apply it to ps df:

km = udf(distance, FloatType())

Wall time: 0 ns


In [31]:
%%time

#apply function to create distance col ps:
df = df.withColumn('distance_km', km(df.start_station_latitude, df.start_station_longitude, df.end_station_lat,
                                 df.end_station_long))

Wall time: 39.9 ms


In [48]:
#test:
df.select('distance_km', 'dur_min').show()

+-----------+---------+
|distance_km|  dur_min|
+-----------+---------+
| 0.71572316| 4.383333|
|  3.3172247|17.633333|
|  1.6952463|10.283334|
|   0.885981|      4.4|
|  2.3261278|12.333333|
|  0.6012847|      3.9|
|  1.5424327|6.0833335|
|  1.2706691| 8.966666|
|   2.025812|14.633333|
|  1.7909632|     13.1|
|        0.0|334.11667|
|   2.423492|12.566667|
|   2.132937|    10.55|
|  1.9964571|27.166666|
|  1.4335872|77.833336|
|  0.7011822| 8.233334|
|  0.7011822|     3.95|
|  0.7011822|39.933334|
|  1.9563731|    35.85|
|  1.0654027| 9.616667|
+-----------+---------+
only showing top 20 rows



In [32]:
%%time

#change dtype:

cols = ['start_station_longitude','start_station_latitude', 'end_station_long','end_station_lat']
for col_name in cols:
    kdf[col_name] = kdf[col_name].astype(float)

Wall time: 887 ms


In [38]:
%%time

#apply function to create distance column in kdf:
kdf['distance_km'] = kdf.apply(lambda i: distance(i['start_station_longitude'],i['start_station_latitude'],
                            i['end_station_long'],i['end_station_lat']), axis=1)

Wall time: 1min 14s


In [58]:
#test:
kdf[['distance_km', 'dur_min']].head()

,distance_km,dur_min
26,3.459368,10.650000
29,1.416776,10.650000
474,2.147084,10.650000
964,1.431382,10.650000
17179869487,1.214015,13.266667


In [39]:
%%time

#calc hours ps:
df = df.withColumn('dur_hrs', col('dur_min') / 60)
#change new col to float:
df = df.withColumn('dur_hrs', df.dur_min.cast('float'))

Wall time: 34.9 ms


In [45]:
%%time

#calc hrs kdf:
kdf['dur_hrs'] = kdf['dur_min'] / 60
#change col to float:
kdf['dur_hrs'] = kdf.dur_hrs.astype(float)

Wall time: 573 ms


In [50]:
#create a udf to round off agg values in df:

from pyspark.sql.functions import pandas_udf, PandasUDFType

@pandas_udf("float", PandasUDFType.GROUPED_AGG)
def sum_udf(v):
    return round(v.sum(), 2)
spark.udf.register("sum_udf", sum_udf)

<function __main__.sum_udf(v)>

In [52]:
%%time


#sum of hrs travelled and distance each weekday ps:

#GroupBy and sum multiple columns:
df.groupBy("weekday") \
    .agg({"dur_hrs": 'sum_udf',"distance_km": 'sum_udf'}) \
    .show()

+---------+----------------+--------------------+
|  weekday|sum_udf(dur_hrs)|sum_udf(distance_km)|
+---------+----------------+--------------------+
|Wednesday|       4050046.5|           509488.38|
|  Tuesday|       4063969.5|           512289.47|
|   Friday|       3995384.2|            464876.7|
| Thursday|       4078651.8|           504538.38|
| Saturday|       3510188.8|           271642.47|
|   Monday|       3842117.8|           470992.88|
|   Sunday|       3089187.5|           231227.31|
+---------+----------------+--------------------+

Wall time: 2min 33s


In [51]:
#%%time

#kdf.groupby('dow')[['dur_hrs', 'distance_km']].sum()

In [56]:
%%time

#sum of hrs travelled and distance each weekday kdf:
kdf.groupby('dow').agg({'dur_hrs': 'sum', 'distance_km': 'sum'}).round(2)# this computation is wrong

Wall time: 328 ms


,dur_hrs,distance_km
dow,,
Wednesday,1565.05,13258.42
Tuesday,1538.46,13146.52
Friday,1355.92,11345.17
Thursday,1522.83,12889.84
Saturday,751.81,5873.25
Monday,1429.80,12164.80
Sunday,652.83,5019.99


In [ ]:
#filter:

In [67]:
%%time

#create subset for males and females ps:

df_males = df.filter(df.member_gender == 'Male').show()
df_females = df.filter(df.member_gender == 'Female').show()

+-------+-------+--------------------+--------------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+---------------+----------------+----------+-----------------+-------------+-----------------------+----------+-----------+---------+----------+---------+------------------+-----------+---------+
|bike_id|dur_sec|          start_time|            end_time|start_station_id|  start_station_name|start_station_latitude|start_station_longitude|end_station_id|    end_station_name|end_station_lat|end_station_long| user_type|member_birth_year|member_gender|bike_share_for_all_trip|member_age|  age_group|  dur_min|  start_dt|  weekday|      avg(dur_min)|distance_km|  dur_hrs|
+-------+-------+--------------------+--------------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+---------------+----------------+----------+-----------------+------

+-------+-------+--------------------+--------------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+---------------+----------------+----------+-----------------+-------------+-----------------------+----------+-----------+---------+----------+---------+------------------+-----------+---------+
|bike_id|dur_sec|          start_time|            end_time|start_station_id|  start_station_name|start_station_latitude|start_station_longitude|end_station_id|    end_station_name|end_station_lat|end_station_long| user_type|member_birth_year|member_gender|bike_share_for_all_trip|member_age|  age_group|  dur_min|  start_dt|  weekday|      avg(dur_min)|distance_km|  dur_hrs|
+-------+-------+--------------------+--------------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+---------------+----------------+----------+-----------------+------

In [70]:
%%time

#create subset males and females kdf:
kdf_males = kdf[kdf.member_gender == 'Male']
kdf_females = kdf[kdf.member_gender == 'Female']

Wall time: 828 ms


In [71]:
#test males ps:
df_males.select('dur_hrs', 'distance_km').show()

AttributeError: 'NoneType' object has no attribute 'select'

In [72]:
kdf_males[['dur_hrs', 'distance_km']].head(20)

,dur_hrs,distance_km
26,0.177500,3.459368
29,0.177500,1.416776
474,0.177500,2.147084
964,0.177500,1.431382
17179869487,0.221111,1.214015
17179870343,0.221111,2.176397
214748365609,0.208333,1.150987
214748365806,0.208333,1.455958
214748365895,0.208333,1.168009
214748365962,0.208333,2.461159


In [ ]:
#trim whitespaces columns: